*Authors:* Ismael Mendoza, Andrii Torchylo

*Outline*

- How to use each deblender and example detections or reconstructions (plots)

- At least one example of customizing the hyper-parameters of a given deblender and how that changes output

- How to run multiple deblenders simultaneously on the same data (sep + peak_finder)